In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import json

In [2]:
tmp1 = pd.read_csv("..//data/weedmaps_joined.csv")

- is there many licenses that have conflicing 
- create a "licenses_not_joined" file
 - collect statistics on how many licenses joined correctly with the adult-use / medicinal column.

In [10]:
pd.crosstab(tmp1["license_type"], tmp1["Adult-Use/Medicinal"])

Adult-Use/Medicinal,Adult-Use,BOTH,Medicinal
license_type,,,
hybrid,27,313,1
medical,2,40,26
recreational,14,37,0


In [5]:
master_list = pd.read_csv("..//data/searchResults.csv")
left_to_merge = pd.read_csv("..//data/weedmaps_not_joined.csv")

## Clean data a bit
- remove hyphen
- get zip code from master list
- get city from Premise Address

In [6]:
def remove_symbol(row, col, string, string_repl):
    if pd.isnull(row[col]):
        return ""
    
    return re.sub("\(|\)|\s+|\.", "", row[col].replace(string, string_repl)).strip()

In [7]:
left_to_merge.phone = left_to_merge.apply(lambda row: remove_symbol(row, "phone", "-", ""), axis = 1)

In [8]:
def break_fields(row, col, string):
    if pd.isnull(row[col]):
        return ""
    
    test = re.search(string, row[col], flags = re.I)
    if test != None:
        return test.group()
    
    return ""

In [9]:
master_list["zip_code"] = master_list.apply(lambda row: break_fields(row, "Premise Address", "([0-9]{5})|([0-9]{9})"), axis = 1)

In [10]:
def get_city(row, col):
    street_abbrev = [" w ", " blvd ", " st ", " rd ", " pkwy ", " ave ", 
                     " ctr ", " cir ", " ct ", " dr ", " ln ", " lk ", 
                     " lp ", " pl ", " sq ", " tr ", " e ", " n ", " s ", 
                     " hwy ", " way ", " wy ", ]
    fix = {"[0-9]": "", "\s+": " ", "san fran": "san francisco", "avenue": "", "highway": ""}
    if pd.isnull(row[col]):
        return ""
    
    left_limit = 0
    check_list = []
    for i in street_abbrev:
        street = re.search(i, row[col], flags = re.I)
        if street != None:  
            check_list.append(street.end())
        else:
            check_list.append(0)
    left_limit = max(check_list)
    
    right_limit = re.search(", ", row[col], flags = re.I)
    if right_limit != None:
        right_limit = right_limit.start()
        
    tmp = row[col][left_limit: right_limit].strip()
    
    for i in fix:
        tmp = re.sub(i, fix[i], tmp, flags = re.I)
        
    return tmp.strip()

In [11]:
master_list["city"] = master_list.apply(lambda row: get_city(row, "Premise Address"), axis = 1).str.lower()

In [74]:
def get_email(row, col):
    replace = {"email": "", "Email": "", "-": ""}
    if pd.isnull(row[col]):
        return ""
    check = re.search("email.{,1}?-.+?@.+?\s", row[col], flags = re.I)
    
    if check is not None:
        check = check.group()
        for i in replace:
            check = check.replace(i, replace[i])
        return check
    
    return ""

def get_website(row, col):
    if pd.isnull(row[col]):
        return ""
    check = re.search("website-", row[col], flags = re.I)
    
    if check is not None:
        check = row[col][check.end():]
        return check.strip()
    
    return ""

def get_name(row, col):
    if pd.isnull(row[col]):
        return ""
    check = re.search(":", row[col], flags = re.I)
    
    if check is not None:
        check = row[col][:check.end() - 1]
        return check.strip()
    
    return ""

In [75]:
master_list["email"] = master_list.apply(lambda row: get_email(row, "Business Contact Information"), axis = 1)
master_list["website"] = master_list.apply(lambda row: get_website(row, "Business Contact Information"), axis = 1)
master_list["company_name"] = master_list.apply(lambda row: get_name(row, "Business Contact Information"), axis = 1)

In [76]:
master_list

,License Number,License Type,Business Owner,Business Contact Information,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,zip_code,city,email,website,company_name
0,C11-18-0000939-TEMP,Cannabis - Distributor Temporary License,Angel Fernandez,MOVOCAN : Email- medigroupangel@gmail.com : Ph...,Corporation,"CALEXICO, CA 922319530 County: IMPERIAL",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,92231,calexico,medigroupangel@gmail.com,www.movocan.com,MOVOCAN
1,C10-18-0000287-TEMP,Cannabis - Retailer Temporary License,Angel Fernandez,MOVOCAN : Email- medigroupangel@gmail.com : Ph...,Corporation,"1632 GATEWAY RD CALEXICO, CA 922319530 County:...",Active,07/29/2019,12/31/2018,N/A for this license type,Medicinal,92231,calexico,medigroupangel@gmail.com,www.movocan.com,MOVOCAN
2,C9-18-0000229-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Angel Fernandez,MOVOCAN : Email- medigroupangel@gmail.com : Ph...,Corporation,"CALEXICO, CA 922319530 County: IMPERIAL",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,92231,calexico,medigroupangel@gmail.com,www.movocan.com,MOVOCAN
3,C12-18-0000217-TEMP,Cannabis - Microbusiness Temporary License,Javier Armas,Javier Armas : Javier's Organics : Email- Jav...,Sole Proprietorship,"OAKLAND, CA 946012132 County: ALAMEDA",Active,07/29/2019,12/31/2018,"Level 1 Manufacturer, Retailer Non-Storefront,...",BOTH,94601,oakland,Javiersorganics@gmail.com,,Javier Armas
4,C9-18-0000230-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,martha Medina,RGG LLC : Ruby's Green Garden : Email- rubysg...,Limited Liability Company,"OAKLAND, CA 946012132 County: ALAMEDA",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,94601,oakland,rubysgreengarden.inc@gmail.com,,RGG LLC
5,C11-18-0000940-TEMP,Cannabis - Distributor Temporary License,Frank Walker,Dubz Garden Supply LLC : Email- frank.walker@y...,Limited Liability Company,"OAKLAND, CA 946012850 County: ALAMEDA",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,94601,oakland,frank.walker@yahoo.com,,Dubz Garden Supply LLC
6,C11-18-0000941-TEMP,Cannabis - Distributor Temporary License,David Nola,"LONG BEACH HOLISTIC HEALTH, LLC : Email- Socal...",Limited Liability Company,"LONG BEACH, CA 908131131 County: LOS ANGELES",Active,07/29/2019,12/31/2018,N/A for this license type,Medicinal,90813,long beach,Socalmurph@gmail.com,,"LONG BEACH HOLISTIC HEALTH, LLC"
7,C10-18-0000288-TEMP,Cannabis - Retailer Temporary License,Basil Bailey,ORIGINAL BALBOA CAREGIVERS : Email- balboacare...,Corporation,"9419 MASON AVE CHATSWORTH, CA 913115204 County...",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,91311,chatsworth,balboacaregivers@yahoo.com,,ORIGINAL BALBOA CAREGIVERS
8,C11-18-0000942-TEMP,Cannabis - Distributor Temporary License,jia zhu,UNIVERSAL CANNABIS SOLUTION INC : Email- unive...,Corporation,"OAKLAND, CA 946214401 County: ALAMEDA",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,94621,oakland,universalcannabissolution@gmail.com,www.calcannabisgarden.com,UNIVERSAL CANNABIS SOLUTION INC
9,C13-18-0000159-TEMP,Cannabis - Distributor-Transport Only Temporar...,Sheldon Norberg,TRICHOMETRY INC. : Email- info@trichometry.com...,Corporation,"REDWAY, CA 95560 County: HUMBOLDT",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,95560,redway,info@trichometry.com,www.trichometry.com,TRICHOMETRY INC.


In [18]:
def search_column(row, col, string):
    if pd.isnull(row[col]):
        return False
    elif re.search(string, row[col], flags = re.I) != None:
        return True
    else:
        return False

In [19]:
def update_matches(test, matches, num):
    
    for match in test.index:
        if match not in matches:
            matches.update({match: num})
        else:
            matches[match] += num
    
    return matches

In [21]:
def suggest_stores(left_to_merge, master_list):
    
    tmp = 0
    ranked_indexes = {}
    unique_cities = master_list.city.unique()

    for index in left_to_merge.id:
        comparison = left_to_merge[left_to_merge.id == index]
        matches = {}

        # check phone
        #print(ranked_indexes)
        print(comparison.phone.iloc[0])
        print(comparison.zip_code.iloc[0])
        print(comparison.city.iloc[0])
        #print()
        
        test = master_list[master_list.apply(lambda row: search_column(row, "Business Contact Information", comparison.phone.iloc[0]), axis = 1)]    
        matches = update_matches(test, matches, 2)

        # check zip
        test = master_list[master_list.apply(lambda row: search_column(row, "zip_code", comparison.zip_code.iloc[0]), axis = 1)]    
        matches = update_matches(test, matches, 1)

        # check city
        # first find closest city wrt levenschtein distance
        best_city = sorted({x: nltk.edit_distance(comparison.city.iloc[0].lower(), x) for x in unique_cities}.items(), key = lambda x: x[1])[0][0]
        #print(best_city)
        # then find those indices
        test = master_list[master_list.apply(lambda row: search_column(row, "city", best_city), axis = 1)]    
        matches = update_matches(test, matches, 1)

        # sort matches
        ranked_indexes.update({index: sorted(matches.items(), key = lambda x: x[1], reverse = True)})
        
        tmp += 1
        if tmp > 20:
            break

    return ranked_indexes    

In [22]:
all_matches = suggest_stores(left_to_merge, master_list)

0000000000
94044
Pacifica
7072102271
94559
Napa
7076525018
94590
Vallejo
5107762965
94841
Richmond
5109251266
94608
Emeryville


KeyboardInterrupt: 

In [253]:
with open("..//data//matches.json", "w") as f:
    json.dump(all_matches, f)

## match email

In [5]:
left_to_merge.head()

,Unnamed: 0,Unnamed: 0.1,id,name,state,city,latitude,longitude,license_type,address,...,retailer_services,phone,adult_use_cultivation,adult_use_nonstorefront,adult_use_retail,distributor,medical_cultivation,medical_nonstorefront,medical_retail,microbusiness
0,11,11,50268,Lytt,CA,Pacifica,37.634507,-122.491800,medical,1905 Palmetto Ave. Unit B,...,storefront,0000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14,14,60567,Harvest of Napa (Newly Opened),CA,Napa,38.297243,-122.301418,medical,2449 Second Street,...,storefront,707.210.2271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15,15,49957,Mendocino Organics,CA,Vallejo,38.109695,-122.253425,medical,539 A Tennessee ST,...,storefront,(707)652-5018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16,16,20208,Fly Fast Delivery - Richmond,CA,Richmond,37.964331,-122.318325,medical,NaN,...,delivery,(510) 776-2965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22,22,57685,East Bay Therapeutics,CA,Emeryville,37.838307,-122.290292,hybrid,5745 Peladeau Street,...,storefront,(510) 925-1266,NaN,NaN,A100000368,NaN,NaN,NaN,M100000420,NaN


In [93]:
master_list[master_list.apply(lambda row: search_column(row, "Business Contact Information", "vhhc420@gmail.com"), axis = 1)]

,License Number,License Type,Business Owner,Business Contact Information,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,zip_code
1869,C11-18-0000083-TEMP,Cannabis - Distributor Temporary License,Greogry Schoepp,"VHHC, LLC : Email- vhhc420@gmail.com : Phone- ...",Limited Liability Company,"VALLEJO, CA 945904430 County: SOLANO",Active,06/11/2019,08/15/2018,N/A for this license type,BOTH,94590


## match phone

In [90]:
master_list[master_list.apply(lambda row: search_column(row, "Business Contact Information", "7076525018"), axis = 1)]

,License Number,License Type,Business Owner,Business Contact Information,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,zip_code
1869,C11-18-0000083-TEMP,Cannabis - Distributor Temporary License,Greogry Schoepp,"VHHC, LLC : Email- vhhc420@gmail.com : Phone- ...",Limited Liability Company,"VALLEJO, CA 945904430 County: SOLANO",Active,06/11/2019,08/15/2018,N/A for this license type,BOTH,94590


## match zip

In [72]:
master_list[master_list.apply(lambda row: search_column(row, "zip_code", "94903"), axis = 1)]

,License Number,License Type,Business Owner,Business Contact Information,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,zip_code
1544,C9-18-0000070-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Alexa Wall,"LUMALIGN, LLC : Moonflower : Email- alexa@moo...",Limited Liability Company,"SAN RAFAEL, CA 949031700 County: MARIN",Active,09/15/2019,11/19/2018,N/A for this license type,Medicinal,94903


## match city

In [74]:
master_list[master_list.apply(lambda row: search_column(row, "Premise Address", "Marin"), axis = 1)]

,License Number,License Type,Business Owner,Business Contact Information,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,zip_code
154,C13-18-0000156-TEMP,Cannabis - Distributor-Transport Only Temporar...,Albert da Cunha,IBALD Enterprises : Marin Medicinals : Email-...,Limited Liability Company,"SAN RAFAEL, CA 94901 County: MARIN",Active,07/27/2019,12/29/2018,N/A for this license type,Medicinal,94901
742,C13-18-0000113-TEMP,Cannabis - Distributor-Transport Only Temporar...,Joaquin Lujan,Tamalpais Co-Packing : Email- joaquintlujan@gm...,Limited Liability Company,"SAN RAFAEL, CA 949014720 County: MARIN",Active,07/18/2019,12/20/2018,N/A for this license type,Medicinal,94901
769,C11-18-0000603-TEMP,Cannabis - Distributor Temporary License,Monica Barker,"NICE GUYS DISTRIBUTION, INC. : Email- monica@n...",Corporation,"SAN RAFAEL, CA 949015374 County: MARIN",Active,07/17/2019,12/19/2018,N/A for this license type,Medicinal,94901
1184,C9-18-0000104-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Nourit Rafael,BUTTERCUP & SPRING : ONA.life : Email- Nurit@...,Corporation,"SAN RAFAEL, CA 94901 County: MARIN",Active,07/08/2019,12/10/2018,N/A for this license type,Medicinal,94901
1544,C9-18-0000070-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Alexa Wall,"LUMALIGN, LLC : Moonflower : Email- alexa@moo...",Limited Liability Company,"SAN RAFAEL, CA 949031700 County: MARIN",Active,09/15/2019,11/19/2018,N/A for this license type,Medicinal,94903
1741,C9-18-0000043-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Emily Elam,FIREFLY MARIN : Firefly Marin : Email- emily....,Limited Liability Company,"San Rafael, CA 94901 County: MARIN",Active,08/01/2019,10/05/2018,N/A for this license type,Medicinal,94901
1808,C11-18-0000102-TEMP,Cannabis - Distributor Temporary License,George Bianchini,LIPOSOME FORMULATIONS .INC : Email- niaid@well...,Corporation,"NOVATO, CA 94949 County: MARIN",Active,07/02/2019,09/05/2018,N/A for this license type,Medicinal,94949
2043,C9-18-0000005-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Robert Elam,ACCESS MARIN : California Wellness : Email- e...,Limited Liability Company,"SAN RAFAEL, CA 949015571 County: MARIN",Canceled,10/11/2018,06/13/2018,N/A for this license type,Medicinal,94901
2108,M9-18-0000105-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Brian Bjork,THE BJORK GROUP : MARIN GARDENS : Email- bria...,Corporation,"SAN RAFAEL, CA 94901 County: MARIN",Active,06/16/2019,05/22/2018,N/A for this license type,BOTH,94901
2280,M9-18-0000091-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Scott Perkins,"CW GROUP, INC. : California Wellness : Email-...",Corporation,"SAN RAFAEL, CA 949015571 County: MARIN",Expired,08/11/2018,04/13/2018,N/A for this license type,Medicinal,94901
